In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt
summer_refractive_index = 1+0.00020409512542851616
winter_refractive_index = 1+0.00020953240736654166
import copy 
import sys
import glob
from datetime import datetime

import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as stats
from scipy.interpolate import interp1d
import statsmodels.api as sm

import matplotlib.pyplot as plt
from ctapipe.io import EventSource
from ctapipe.image import ImageProcessor, tailcuts_clean
from ctapipe.calib import CameraCalibrator
from ctapipe.visualization import CameraDisplay
from astropy.table import Table, vstack

#from muonpipe import usefull_func
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
filename = '/Users/vdk/muons2024//simtel_files/2024year_tuned_nooulier_reflectivity_additional/run101_muon.simtel.gz'
source = EventSource(filename, max_events = 1)
camgeom = source.subarray.tel[1].camera.geometry

start_date_2019 = datetime.strptime("2019-11-25 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()

start_date_2020 = datetime.strptime("2020-01-01 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()
end_date_2020 = datetime.strptime("2020-12-31 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()

start_date_2021 = datetime.strptime("2021-01-01 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()
end_date_2021 = datetime.strptime("2021-12-31 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()

start_date_2022 = datetime.strptime("2022-01-01 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()
end_date_2022 = datetime.strptime("2022-12-31 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()

start_date_2023 = datetime.strptime("2023-01-01 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()
end_date_2023 = datetime.strptime("2023-12-31 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()

start_date_2024 = datetime.strptime("2024-01-01 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()
end_date_2024 = datetime.strptime("2024-12-31 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()

size_outside_cut = 500
muon_efficiency_cut = 1
min_impact = 2.2199933748101555
max_impact = 9.983608702234397


In [ ]:
listdir= glob.glob('/Users/vdk/muons2024/psf_work/focus_offset_0/tables/*')

dat = Table.read('{}'.format(listdir[0]), format='fits')

for muon_file in listdir:
    dat2 = Table.read(muon_file, format='fits')

    if 'good_ring' in dat.colnames:
        dat['good_ring'] = dat['good_ring'].astype(bool)
    if 'is_valid' in dat2.colnames:
        dat2['is_valid'] = dat2['is_valid'].astype(bool)
    if 'parameters_at_limit' in dat2.colnames:
        dat2['parameters_at_limit'] = dat2['parameters_at_limit'].astype(bool)       
    if 'good_ring' in dat2.colnames:
        dat2['good_ring'] = dat2['good_ring'].astype(bool)
    
    # Stack the tables
    if dat is None:
        dat = dat2
    else:
        dat = vstack([dat, dat2])

df = dat.to_pandas()
df_sim_focus_offset0 = df[
    df['good_ring'] & 
    (df['muon_efficiency'] < muon_efficiency_cut) & 
    (df['size_outside'] < size_outside_cut)
]

df_sim_focus_offset0

In [ ]:
listdir= glob.glob('/Users/vdk/muons2024/lapalma_simulations/proper_mc_config/nsbtune2024year_tuned_reflectivity_0015alignment/tables/*')

dat = Table.read('{}'.format(listdir[0]), format='fits')

for muon_file in listdir:
    dat2 = Table.read(muon_file, format='fits')

    if 'good_ring' in dat.colnames:
        dat['good_ring'] = dat['good_ring'].astype(bool)
    if 'is_valid' in dat2.colnames:
        dat2['is_valid'] = dat2['is_valid'].astype(bool)
    if 'parameters_at_limit' in dat2.colnames:
        dat2['parameters_at_limit'] = dat2['parameters_at_limit'].astype(bool)       
    if 'good_ring' in dat2.colnames:
        dat2['good_ring'] = dat2['good_ring'].astype(bool)
    
    # Stack the tables
    if dat is None:
        dat = dat2
    else:
        dat = vstack([dat, dat2])

df = dat.to_pandas()
df_sim_0015_alignment = df[
    df['good_ring'] & 
    (df['muon_efficiency'] < muon_efficiency_cut) & 
    (df['size_outside'] < size_outside_cut)
]

df_sim_0015_alignment

In [ ]:
winter_good_data_2024 = glob.glob('/Users/vdk/muons2024/data_quality_winter_2024/tables/*')

dat = Table.read('{}'.format(winter_good_data_2024[0]), format='fits')

for muon_file in winter_good_data_2024:
    dat2 = Table.read(muon_file, format='fits')

    if 'good_ring' in dat.colnames:
        dat['good_ring'] = dat['good_ring'].astype(bool)
    if 'is_valid' in dat2.colnames:
        dat2['is_valid'] = dat2['is_valid'].astype(bool)
    if 'parameters_at_limit' in dat2.colnames:
        dat2['parameters_at_limit'] = dat2['parameters_at_limit'].astype(bool)       
    if 'good_ring' in dat2.colnames:
        dat2['good_ring'] = dat2['good_ring'].astype(bool)
    
    # Stack the tables
    if dat is None:
        dat = dat2
    else:
        dat = vstack([dat, dat2])

df = dat.to_pandas()

df_good_data_winter_2024 = df[
    df['good_ring'] & 
    (df['muon_efficiency'] < muon_efficiency_cut) & 
    (df['size_outside'] < size_outside_cut)
]

df_good_data_winter_2024

In [ ]:
ring_completeness_cut = 0.99
ring_containment_cut = 0
min_radius_cut = 1
max_radius_cut = 1.2
ring_center_distance_cut = 2
impact_distance_cut = 15

def calculate_survival_percentage(df, df_cut):
    return (len(df_cut) / len(df)) * 100

def apply_cuts(df, completeness_cut, containment_cut, min_radius_cut, max_radius_cut, center_distance_cut, impact_distance_cut):
    return df[
        (df['ring_completeness'] > completeness_cut) &
        (df['ring_containment'] > containment_cut) &
        (df['ring_radius'] > min_radius_cut) &
        (df['ring_radius'] < max_radius_cut) &
        (df['ring_center_y'].abs() < center_distance_cut) &
        (df['impact_parameter'] < impact_distance_cut)
    ]


# Apply cuts to each dataframe
# df_data_cut = apply_cuts(df_good_data_real, ring_completeness_cut, ring_containment_cut, min_radius_cut, max_radius_cut, ring_center_distance_cut, impact_distance_cut)
# print(f"Events survived for df_data_cut: {len(df_data_cut)} ({calculate_survival_percentage(df_good_data_real, df_data_cut):.2f}%) out of {len(df_good_data_real)}")

df_good_data_winter_2024_cut = apply_cuts(df_good_data_winter_2024, ring_completeness_cut, ring_containment_cut, min_radius_cut, max_radius_cut, ring_center_distance_cut, impact_distance_cut)
print(f"Events survived for df_good_data_winter_2024_cut: {len(df_good_data_winter_2024_cut)} ({calculate_survival_percentage(df_good_data_winter_2024, df_good_data_winter_2024_cut):.2f}%) out of {len(df_good_data_winter_2024)}")

df_sim_focus_offset0_cut = apply_cuts(df_sim_focus_offset0, ring_completeness_cut, ring_containment_cut, min_radius_cut, max_radius_cut, ring_center_distance_cut, impact_distance_cut)
print(f"Events survived for df_sim_focus_offset0_cut: {len(df_sim_focus_offset0_cut)} ({calculate_survival_percentage(df_sim_focus_offset0, df_sim_focus_offset0_cut):.2f}%) out of {len(df_sim_focus_offset0)}")

df_sim_0015_alignment_cut = apply_cuts(df_sim_0015_alignment, ring_completeness_cut, ring_containment_cut, min_radius_cut, max_radius_cut, ring_center_distance_cut, impact_distance_cut)
print(f"Events survived for df_sim_0015_alignment_cut: {len(df_sim_0015_alignment_cut)} ({calculate_survival_percentage(df_sim_0015_alignment, df_sim_0015_alignment_cut):.2f}%) out of {len(df_sim_0015_alignment)}")

In [ ]:
def weighted_data(df, power_index):
    df_copy = copy.deepcopy(df[['ring_radius', 'mc_energy', 'ring_completeness', 'ring_containment', 'impact_parameter']])
    df_copy['weights'] = df_copy['mc_energy'] ** (-power_index)
    return df_copy


df_sim_standard_power_law_weigth = weighted_data(df_sim_0015_alignment_cut, 0.7)
df_sim_focus_offset0_weight = weighted_data(df_sim_focus_offset0_cut, 0.7)


In [ ]:
ring_completeness_cut_hist = 0
bin_number = 100
density_param = True

min_value = min_radius_cut
max_value = max_radius_cut

# Define the bin edges
bins = np.linspace(min_value, max_value, bin_number)  

plt.figure(figsize=(11,8))

z,x,c = plt.hist(df_sim_0015_alignment_cut['ring_radius'], bins=bins, label='Simulation with 0.015 deg alignment', alpha=0.7, density=density_param, histtype='step', lw=3)

z,x,c = plt.hist(df_sim_focus_offset0_cut['ring_radius'], bins=bins, label='Focus offset 0', alpha=0.7, density=density_param, histtype='step', lw=3) 

z,x,c = plt.hist(df_good_data_winter_2024_cut['ring_radius'][df_good_data_winter_2024_cut['ring_completeness'] > ring_completeness_cut_hist], bins=bins, label='Winter 2024 + zenith angle < 20 deg + good atmo conditions',
         alpha=0.5, density=density_param)

z,x,c = plt.hist(df_sim_focus_offset0_weight['ring_radius'], weights=df_sim_focus_offset0_weight['weights'], bins=bins, label='Focus offset 0 weighted', alpha=0.7, density=density_param, histtype='step', lw=3)

plt.xlabel('Ring Radius [deg]')
plt.axvline(1.2157382060313375, lw = 2, ls = '-.', c = 'r', label = 'summer max ring radius on 2 km',alpha=0.75)
plt.axvline(1.2354433340796351, lw = 2, ls = '-.', c = 'k', label = 'winter max ring radius on 2 km',alpha=0.75)
plt.ylabel('Normalized Count')
#plt.title('Histogram of Radius with Reweighted Energy Distribution')
plt.xlim(0.7,1.3)
#plt.ylim(0,10)
plt.legend()
plt.show()

In [ ]:
# Define the number of entries to use from each dataset
MAX_ENTRIES = 2100000
NUM_BINS = 10
COMPLETENESS_CUT = ring_completeness_cut
QUANTILE_CUT = 0.2

# Calculate the 20th percentile (quantile 0.2) for each dataset
quantiles = {
    'sim_0015_alignment': df_sim_0015_alignment_cut['ring_width'].quantile(QUANTILE_CUT),
    'focus_offset0_0015 alignment': df_sim_focus_offset0_cut['ring_width'].quantile(QUANTILE_CUT),
    'data': df_good_data_winter_2024_cut['ring_width'].quantile(QUANTILE_CUT)
}

# Define a list of tuples containing each dataframe, its corresponding quantile, and label
datasets = [
    (df_sim_0015_alignment_cut[df_sim_0015_alignment_cut['ring_completeness'] > COMPLETENESS_CUT], quantiles['sim_0015_alignment'], '0.015 deg'),
    (df_sim_focus_offset0_cut[df_sim_focus_offset0_cut['ring_completeness'] > COMPLETENESS_CUT], quantiles['focus_offset0_0015 alignment'], 'focus_offset0_0015 alignment'),
    (df_good_data_winter_2024_cut[df_good_data_winter_2024_cut['ring_completeness'] > COMPLETENESS_CUT], quantiles['data'], 'Data')
]

# datasets = [
#     (df_sim_00043_cut[df_sim_00043_cut['ring_completeness'] > COMPLETENESS_CUT], quantiles['sim_2024_tune'], '0.0046 deg (Reference)'),
#     (df_sim_001_cut[df_sim_001_cut['ring_completeness'] > COMPLETENESS_CUT], quantiles['sim_001_alignment'], '0.01 deg'),
#     (df_sim_0015_cut[df_sim_0015_cut['ring_completeness'] > COMPLETENESS_CUT], quantiles['sim_0015_alignment'], '0.015 deg'),
#     (df_data_cut[df_data_cut['ring_completeness'] > COMPLETENESS_CUT], quantiles['data'], 'Data')
# ]

# Initialize a dictionary to store binned data for each dataset
binned_data = {}

# Iterate over each dataset to preprocess data
for df, quantile, label in datasets:
    # Limit to the first MAX_ENTRIES entries
    limited_df = df.iloc[:MAX_ENTRIES]
    
    # Filter the data based on the quantile
    filtered_df = limited_df[limited_df['ring_width'] < quantile].copy()
    
    # Check if filtered_df is not empty to avoid errors
    if filtered_df.empty:
        print(f"Warning: No data points below the 20th percentile for '{label}' in the first {MAX_ENTRIES} entries.")
        continue
    
    # Bin 'ring_radius' into NUM_BINS equal-width bins
    filtered_df['radius_bin'] = pd.cut(filtered_df['ring_radius'], bins=NUM_BINS, labels=False)
    
    # Remove any rows with NaN in 'radius_bin' (can occur if 'ring_radius' has values outside the bin range)
    filtered_df = filtered_df.dropna(subset=['radius_bin'])
    
    # Convert 'radius_bin' to integer
    filtered_df['radius_bin'] = filtered_df['radius_bin'].astype(int)
    
    # Group by 'radius_bin' and compute mean 'ring_radius', 'ring_width', and the ratio 'ring_width'/'ring_radius'
    grouped = filtered_df.groupby('radius_bin').agg({
        'ring_radius': 'mean',
        'ring_width': 'mean'
    }).reset_index()
    grouped['width_radius_ratio'] = grouped['ring_width'] / grouped['ring_radius']
    
    # Store the grouped data
    binned_data[label] = grouped

# Plot settings
alpha_reg = 0.7
palette = sns.color_palette("husl", len(binned_data))[:-1] + ['k']

plt.figure(figsize=(12, 10))
flag = '2023-24'

# Iterate over each binned dataset and plot with regression lines and confidence intervals
for i, (label, grouped_df) in enumerate(binned_data.items()):
    sns.regplot(
        x='ring_radius',
        y='width_radius_ratio',
        data=grouped_df,
        label=label,
        scatter_kws={'alpha': alpha_reg, 's': 100},
        line_kws={'alpha': alpha_reg, 'color': palette[i]},
        ci=95,
        color=palette[i],
        robust=True
    )

# Customize plot labels and limits
plt.xlabel('Muon Radius [deg]', fontsize=24)
plt.ylabel('Muon Ring Width / Radius', fontsize=24)
plt.ylim(0.03, 0.08)
#plt.xlim(min_radius_cut, max_radius_cut)
plt.xlim(1,1.2)

# Enhance the legend
#plt.legend(prop={'size': 26}, loc='lower right', frameon=True)
plt.legend(ncol=2, fontsize=26, loc='upper right', frameon=True)

# Add a title for context
#plt.title(f'Ring width vs radius ratio for 2024 year, {QUANTILE_CUT*100}% quantile, completeness > {COMPLETENESS_CUT}', fontsize=18)

# Improve layout and display the plot
plt.tight_layout()
plt.grid(alpha=0.3)
plt.tick_params(axis='both', labelsize=15)

# Optionally, save the figure
#plt.savefig(f'/Users/vdk/muons2024/psf_work/width_radius/widthradius_vs_radius_completeness_{COMPLETENESS_CUT}_quantile{QUANTILE_CUT*100}%_radiusrange{min_radius_cut,max_radius_cut}_center{ring_center_distance_cut}_impact{impact_distance_cut}nsb_tune2024.pdf', dpi=200)
plt.show()

In [ ]:
plt.figure(figsize=(11,8))
z,x,c = plt.hist(df_sim_0015_alignment['mc_energy']*1000,bins = np.linspace(8,1000, 1000), histtype='step', lw=3, label = '0.015 deg alignment', density=True)
z,x,c = plt.hist(df_sim_0015_alignment_cut['mc_energy']*1000,bins = np.linspace(8,1000, 1000), density=True, label='0.015 deg alignment cut', histtype='step', lw=3)
plt.xlim(0,100)
plt.legend()

In [ ]:
z,x,c = plt.hist(df_sim_0015_alignment_cut['mc_energy']*1000,bins = 100)
plt.xlim(0,100)